<div style="text-align: center; font-size: 21px; font-weight: bold; text-decoration: underline;">
    Prediction of future Amazon.com stock prices using Machine Learning algorithms
</div>

Contributors:
- Wiktoria Sarabon
- Martyna Szawłowska
- Magdalena Lamczyk
- Jakub Galikowski
- Natalia Szczepkowska

## Imports

In [11]:
import pandas as pd
import seaborn as sns

## Data overview

In [10]:
data = pd.read_csv('data.csv')
data.head()

,Date,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Volume,SP100_Open,SP100_High,SP100_Low,SP100_Close,SP100_Volume,Inflation_USA,Inflation_EU,Inflation_World,Revenue_Billions
0,1997-05-15,0.121875,0.125000,0.096354,0.097917,1.443120e+09,410.994995,414.295013,409.625000,414.200012,9.163600e+08,2.34,3.11,5.57,0.064587
1,1997-05-16,0.098438,0.098958,0.085417,0.086458,2.940000e+08,414.200012,414.200012,406.279999,406.279999,9.735600e+08,2.34,3.11,5.57,0.064948
2,1997-05-19,0.088021,0.088542,0.081250,0.085417,1.221360e+08,406.279999,410.350006,406.279999,409.230011,6.902800e+08,2.34,3.11,5.57,0.066034
3,1997-05-20,0.086458,0.087500,0.081771,0.081771,1.093440e+08,409.230011,414.480011,405.890015,414.160004,9.017000e+08,2.34,3.11,5.57,0.066395
4,1997-05-21,0.081771,0.082292,0.068750,0.071354,3.770640e+08,414.160004,416.739990,411.035004,413.140015,1.081460e+09,2.34,3.11,5.57,0.066757


The dataset contains the following 16 columns:
- **Date**
- **AMZN_Open**: The price at which the Amazon.com stock first traded upon the opening of the market (in USD),
- **AMZN_High**: The highest price at which the Amazon.com stock traded during the trading day (in USD),
- **AMZN_Low**: The lowest price at which the Amazon.com stock traded during the trading day (in USD),
- **AMZN_Close**: The price at which the Amazon.com stock last traded upon the closing of the market (in USD),
- **AMZN_Volume**: The total number of Amazon.com shares traded during the trading day (in USD),
- **SP100_Open**: The collective starting point of the stocks within the S&P 100 index (in USD),
- **SP100_High**: The peak level of the S&P 100 index's performance throughout the trading session (in USD),
- **SP100_Low**: The lowest point of the S&P 100 index's performance during the trading session (in USD),
- **SP100_Close**: The collective ending point of the stocks within the S&P 100 index (in USD),
- **SP100_Volume**: The overall level of activity and trading interest in the stocks comprising the S&P 100 index (in USD),
- **Inflation_USA**: Annual mean inflation rate in the USA (in %)
- **Inflation_UE**: Annual mean inflation rate in the European Union (in %)
- **Inflation_World**: Annual mean inflation rate in the world (in %)
- **Revenue_Billions**: Amazon.com revenue (in billions USD)

In [16]:
# info o danych, podstawowe statystyki

## Exploratory Data Analysis

In [17]:
# wykresy: pairplot, heatmap, wykresy sp100_close i amzn_close w czasie itp.

# Prediction of future Amazon.com stock prices

## Linear Regression

In [18]:
# lm models

## SVM (Support Vector Machines)

In [19]:
# svm models

## Random Forest Regression

In [20]:
# rfr models

## Gradient Boosting Regressor

In [21]:
# gbr models

## LSTM (Long-Short Term Memory)

In [22]:
# lstm models

## GRU (Gated Recurrent Units)

In [23]:
# gru models

## CNN (Convolutional Neural Network)

In [24]:
# cnn models

## ARIMA (Auto-Regressive Integrated Moving Average)

In [25]:
# arima models

# Best model identification

In [26]:
# the best model is: (to be continued...)

# Real-time data model application

In [27]:
# application of the best model to the real time data (january-may 2024)

# What does the future hold...?

In [28]:
# prediction for the next 2 years